In [1]:
import numpy as np
from faker import Faker
from dotenv import load_dotenv
import os
load_dotenv();

In [2]:
faker = Faker(seed=666)
np.random.seed(666)

In [3]:
keywords = [
    'archeology',
    'museums',
    'music',
    'art',
    'cinema',
    'countryside',
    'tracking',
    'rafting',
    'history',
    'literature',
    'sport',
    'food',
    'wine',
    'beer'
]

In [37]:
def generate_guide():
    guide = {}
    guide['gender'] = np.random.choice(['male', 'female'], size=1).item()
    guide['name'] = (faker.first_name_male() if guide['gender'] == 'male' else faker.first_name_female()) + ' ' + faker.last_name()
    guide['birth_date'] = faker.date_of_birth()
    guide['now_available'] = bool(np.random.binomial(1, 0.3))
    guide['languages_spoken'] = list(np.random.choice(['english', 'italian', 'french', 'spanish', 'deutsche', 'dutch', 'bulgarian', 'chinese'], size=1+np.random.poisson(lam=1, size=1).item(), replace=False))
    guide['price'] = int(max(10, np.abs(np.random.normal(30, 5))))
    guide['education'] = np.random.choice(['elementary', 'high-school', 'bachelor', 'master', 'phd'], size=1).item()
    guide['biography'] = faker.profile()['job']
    guide['keywords'] = list(np.random.choice(keywords, size=np.random.poisson(2, size=1).item()))
    guide['current_location'] = {
        "lat": np.random.normal(40.3524, 0.01),
        "lon": np.random.normal(18.1732, 0.01)
    }
    return guide

In [29]:
generate_guide()

{'gender': 'male',
 'name': 'Joshua Gonzalez',
 'birth_date': datetime.date(1928, 12, 27),
 'now_available': False,
 'languages_spoken': ['french', 'dutch'],
 'price': 27,
 'education': 'high-school',
 'biography': 'Chiropodist',
 'keywords': ['food', 'literature'],
 'current_location': {'lat': 40.349934, 'lon': 18.171648}}

In [12]:
from datetime import datetime
from elasticsearch import Elasticsearch

client = Elasticsearch(os.getenv('HOST'), api_key=os.getenv('API_KEY'), verify_certs=False)

client.ssl_context = False

/Users/lorenzobenzoni/Documents/university/projects/MULTIDISCIPLINARY/AsTourX/elasticsearch/venv/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [16]:
# Feed data
client.index(index='guides', body=generate_guide())

/Users/lorenzobenzoni/Documents/university/projects/MULTIDISCIPLINARY/AsTourX/elasticsearch/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'_index': 'guides', '_id': 'pnjQOY8BESXXNo8oZ-sh', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

In [ ]:
N = 200

for i in range(1, N):
    client.index(index='guides', body=generate_guide())
    print(f"Generated {i}/{N} guides")

In [66]:
user = {
    "location": {
        "lat": np.random.normal(40.3524, 0.01),
        "lon": np.random.normal(18.1732, 0.01)
        },
    "keywords": list(np.random.choice(keywords, size=np.random.poisson(2, size=1).item())),
    "languages": ["italian"]
}

user

{'location': {'lat': 40.361688261485256, 'lon': 18.187273390487693},
 'keywords': ['museums', 'tracking'],
 'languages': ['italian']}

In [82]:
query = client.search(index='guides', body={
    "query": {
        "bool": {
            "should": [
                # user and guide should be close to each other
                {
                    "geo_distance": {
                        "distance": "800m",
                        "current_location": user["location"],
                    }
                },
            ],
            "filter": {
                # guide must speak at least one of the user's languages
                "terms": {
                    "languages_spoken": user["languages"]
                },
            },
            "must" :{
                "term": {
                    "now_available": True
                }
            }
        },
    }
})

query.body['hits']['hits'][-1]

/Users/lorenzobenzoni/Documents/university/projects/MULTIDISCIPLINARY/AsTourX/elasticsearch/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'_index': 'guides',
 '_id': '9XnnOY8BESXXNo8oQRRD',
 '_score': 1.3001167,
 '_source': {'gender': 'male',
  'name': 'Nathaniel Stone',
  'birth_date': '1935-07-19',
  'now_available': True,
  'languages_spoken': ['english', 'italian'],
  'price': 30,
  'education': 'bachelor',
  'biography': 'Education officer, community',
  'keywords': ['beer', 'archeology', 'sport', 'cinema', 'rafting'],
  'current_location': {'lat': 40.34773745731428, 'lon': 18.15850933058771}}}